In [1]:
import pandas as pd
import tool
import re
pd.set_option('display.max_columns',500)
pd.set_option('display.width',1000)

In [2]:
token_len = 4
year_word = '道光二十八年'

In [3]:
# token_len = 
# start = 0
# end = 1
# pd.set_option('display.width',1000)

In [4]:
df = pd.read_csv("f_jiashu.csv")

In [5]:
df[:2]

,0,1,2
0,曾国藩家书,道光二十一年五月十八日,\n 儿子国藩跪着禀告\n 父亲大人万福金安。自从闰三月十四日，在京城城门拜送父...
1,曾国藩家书,道光二十二年一月二十六日,\n 儿子国藩跪着禀告\n 父母亲大人万福金安，十月二十二日，收到手谕，敬悉一切...


In [6]:
df.loc[:,'2'] = df.loc[:,'2'].apply(lambda x:x.replace("\n","").replace(" ",""))
df[:2]

,0,1,2
0,曾国藩家书,道光二十一年五月十八日,儿子国藩跪着禀告父亲大人万福金安。自从闰三月十四日，在京城城门拜送父亲回家，后来共接到家信五...
1,曾国藩家书,道光二十二年一月二十六日,儿子国藩跪着禀告父母亲大人万福金安，十月二十二日，收到手谕，敬悉一切。郑小栅那里，小小嫌隙已...


In [7]:
docs = df['2'].values   
res = df.copy()
res[:1]

,0,1,2
0,曾国藩家书,道光二十一年五月十八日,儿子国藩跪着禀告父亲大人万福金安。自从闰三月十四日，在京城城门拜送父亲回家，后来共接到家信五...


In [8]:
res['tokens'] = res['2'].apply(lambda x:tool.ngram(x,token_len))
cur_all_token = res['tokens'].apply(pd.Series).unstack().reset_index(drop=True)
cur_all_tokens = pd.DataFrame(cur_all_token)
cur_all_tokens['cur_all_token_cnt'] = 1
cur_all_tokens = cur_all_tokens.groupby(by=0).sum().reset_index()

cur_all_tokens['token_len'] = cur_all_tokens[0].apply(len)
cur_all_tokens = cur_all_tokens[cur_all_tokens['token_len'] == token_len].reset_index(drop=True)

cur_all_tokens = cur_all_tokens.drop(columns=['token_len'])

cur_all_tokens[:1]

,0,cur_all_token_cnt
0,(木云),1


In [9]:
len(cur_all_tokens)

117008

In [10]:
res['l_tokens'] = res['2'].apply(lambda x: tool.ngram(x,token_len+1))
l_all_token = res['l_tokens'].apply(pd.Series).unstack().reset_index(drop=True)
l_all_tokens = pd.DataFrame(l_all_token)
l_all_tokens['l_tokens_cnt'] = 1
l_all_tokens = l_all_tokens.groupby(by=0).sum().reset_index()

l_all_tokens['token_len'] = l_all_tokens[0].apply(len)
l_all_tokens = l_all_tokens[l_all_tokens['token_len'] == token_len+1].reset_index(drop=True)

l_all_tokens['prefix_token'] = l_all_tokens[0].apply(lambda x:x[:token_len])
l_all_tokens['suffix_token'] = l_all_tokens[0].apply(lambda x:x[1:])

l_all_tokens[:1]

,0,l_tokens_cnt,token_len,prefix_token,suffix_token
0,(木云)台,1,5,(木云),木云)台


In [11]:
prefix_token = pd.DataFrame()
prefix_token['prefix_token'] = l_all_tokens['prefix_token']
prefix_token['prefix_cnt'] = l_all_tokens['l_tokens_cnt']

prefix_token = prefix_token.groupby('prefix_token').max().reset_index()

prefix_token[:1]

,prefix_token,prefix_cnt
0,(木云),1


In [12]:
suffix_token = pd.DataFrame()
suffix_token['suffix_token'] = l_all_tokens['suffix_token']
suffix_token['suffix_cnt'] = l_all_tokens['l_tokens_cnt']

suffix_token = suffix_token.groupby('suffix_token').max().reset_index()

suffix_token[:1]

,suffix_token,suffix_cnt
0,(木云),1


In [13]:
tokens = cur_all_tokens.set_index(0).join(prefix_token.set_index('prefix_token'),how='left')
tokens[:1]

,cur_all_token_cnt,prefix_cnt
0,,
(木云),1,1.0


In [14]:
tokens = tokens.join(suffix_token.set_index('suffix_token'),how='left')
tokens[:1]

,cur_all_token_cnt,prefix_cnt,suffix_cnt
0,,,
(木云),1,1.0,1.0


In [15]:
tokens = tokens.reset_index()
tokens[:1]

,0,cur_all_token_cnt,prefix_cnt,suffix_cnt
0,(木云),1,1.0,1.0


In [16]:
len(tokens)

117008

In [17]:
def is_ok(row):
    token = row[0]
    token_cnt = row['cur_all_token_cnt']
    prefix_cnt = row['prefix_cnt']
    suffix_cnt = row['suffix_cnt']
    #flag = (token_cnt > (prefix_cnt+1)*1.3) and (token_cnt > (suffix_cnt+1)*1.3)
    
    flag = (token_cnt >= prefix_cnt) and (token_cnt >= suffix_cnt)
    if flag:
        flag = token.isalpha()
    return flag

tokens['is_ok'] = tokens.apply(is_ok,axis=1)
tokens = tokens.reset_index(drop=True)
tokens[:1]

,0,cur_all_token_cnt,prefix_cnt,suffix_cnt,is_ok
0,(木云),1,1.0,1.0,False


In [18]:
tokens = tokens[tokens['is_ok']==True]

len(tokens)

60797

In [19]:
tokens = tokens.drop(columns=['prefix_cnt','suffix_cnt','is_ok'])

tokens[:5]

,0,cur_all_token_cnt
4019,一一写了,10
4020,一一写出,1
4021,一一办妥,1
4022,一一向对,1
4023,一一小心,1


In [20]:
cur_year = df[df['1'].str.find(year_word) != -1].reset_index(drop=True)
#cur_year = df[start:end].reset_index(drop=True)
cur_year[:1]

,0,1,2
0,曾国藩家书,咸丰八年三月初六日,沅甫九弟左右：初三日刘福一等回后接来信，知道一切。城里敌军队围困已久，估计不久也可攻下，但要...


In [21]:
cur_year['tokens'] = cur_year['2'].apply(lambda x:tool.ngram(x,token_len))
cur_year_token = cur_year['tokens'].apply(pd.Series).unstack().reset_index(drop=True)
cur_year_tokens = pd.DataFrame(cur_year_token)
cur_year_tokens['cur_year_token'] = 1
cur_year_tokens = cur_year_tokens.groupby(by=0).sum().reset_index()

cur_year_tokens['token_len'] = cur_year_tokens[0].apply(len)
cur_year_tokens = cur_year_tokens[cur_year_tokens['token_len'] == token_len].reset_index(drop=True)

cur_year_tokens = cur_year_tokens.drop(columns=['token_len'])

cur_year_tokens[:1]

,0,cur_year_token
0,·劝捐银,1


In [22]:
cur_year_tokens = cur_year_tokens.set_index(0).join(tokens.set_index(0),how='inner').reset_index()
cur_year_tokens[:5]

,0,cur_year_token,cur_all_token_cnt
0,一下我的,1,1
1,一个人的,1,6
2,一个做一,1,1
3,一个傲字,1,1
4,一个军队,1,1


In [23]:
cur_year_tf = df[df['1'].str.find(year_word) != -1].reset_index(drop=True)
#cur_year_tf = df[start:end].reset_index(drop=True)
cur_year_tf[:1]

,0,1,2
0,曾国藩家书,咸丰八年三月初六日,沅甫九弟左右：初三日刘福一等回后接来信，知道一切。城里敌军队围困已久，估计不久也可攻下，但要...


In [24]:
def word_cnt(x):
    doc_cnt = 0
    if str(x) != 'nan':
        for word in x:
            if word.isalpha():
                doc_cnt += 1
    return doc_cnt
cur_year_tf['word_cnt'] = cur_year_tf['2'].apply(word_cnt)
cur_year_tf[:1]

,0,1,2,word_cnt
0,曾国藩家书,咸丰八年三月初六日,沅甫九弟左右：初三日刘福一等回后接来信，知道一切。城里敌军队围困已久，估计不久也可攻下，但要...,499


In [25]:
cur_year_word_cnt = cur_year_tf['word_cnt'].sum()
cur_year_word_cnt

9356

In [26]:
doc_cnt = len(df)
doc_cnt

195

In [27]:
cur_year_tokens['word_cnt'] = cur_year_word_cnt
cur_year_tokens['doc_cnt'] = doc_cnt
cur_year_tokens[:1]

,0,cur_year_token,cur_all_token_cnt,word_cnt,doc_cnt
0,一下我的,1,1,9356,195


In [28]:
def df_cnt(x,docs):
    cnt = 0
    for doc in docs:
        if doc.find(x) != -1:
            cnt += 1
    return cnt

cur_year_tokens['df_cnt'] = cur_year_tokens[0].apply(lambda x:df_cnt(x,docs))
cur_year_tokens[:1]

,0,cur_year_token,cur_all_token_cnt,word_cnt,doc_cnt,df_cnt
0,一下我的,1,1,9356,195,1


In [29]:
def calc_rate(row):
    word_len = len(row[0])
    cur_year_token = row['cur_year_token']
    cur_all_token_cnt = row['cur_all_token_cnt'] + 1
    
    return 1.0 * cur_year_token / cur_all_token_cnt

cur_year_tokens['rate'] = cur_year_tokens.apply(calc_rate,axis=1)

cur_year_tokens[:1]

,0,cur_year_token,cur_all_token_cnt,word_cnt,doc_cnt,df_cnt,rate
0,一下我的,1,1,9356,195,1,0.5


In [30]:
def calc_tf(row):
    word_len = len(row[0])
    cur_year_token = row['cur_year_token']
    word_cnt = row['word_cnt']
    
    return 1.0 * word_len * cur_year_token / word_cnt

cur_year_tokens['tf'] = cur_year_tokens.apply(calc_tf,axis=1)

cur_year_tokens[:1]

,0,cur_year_token,cur_all_token_cnt,word_cnt,doc_cnt,df_cnt,rate,tf
0,一下我的,1,1,9356,195,1,0.5,0.000428


In [31]:
import math
def calc_idf(row):
    doc_cnt = row['doc_cnt']
    df_cnt = row['df_cnt']
    
    return math.log(1.0 * doc_cnt / df_cnt)

cur_year_tokens['idf'] = cur_year_tokens.apply(calc_idf,axis=1)

cur_year_tokens[:1]

,0,cur_year_token,cur_all_token_cnt,word_cnt,doc_cnt,df_cnt,rate,tf,idf
0,一下我的,1,1,9356,195,1,0.5,0.000428,5.273


In [32]:
def calc_score(row):
    
    rate = row['rate']
    tf = row['tf']
    idf = row['idf']
    
    return rate * tf * idf

cur_year_tokens['score'] = cur_year_tokens.apply(calc_score,axis=1)

cur_year_tokens[:1]

,0,cur_year_token,cur_all_token_cnt,word_cnt,doc_cnt,df_cnt,rate,tf,idf,score
0,一下我的,1,1,9356,195,1,0.5,0.000428,5.273,0.001127


In [33]:
def get_year(x,docs):
    year = ""
    for doc in docs:
        if doc.find(x) != -1:
            times = re.findall('道光.*年.*月.*日|咸丰.*年.*月.*日|同治.*年.*月.*日',doc)
            year += ','.join(times)
    return year
    
def get_cont(x,docs):
    cont = ""
    for doc in docs:
        inx = doc.find(x) 
        if inx != -1:
            cont = doc[max(0,inx-20):inx+20]
            break
    return cont

In [34]:
sort1 = cur_year_tokens.sort_values(by=['score','df_cnt','cur_year_token'],ascending=(False,True,True))
res1 = sort1[:50].reset_index(drop=True)
res1['year'] = res1[0].apply(lambda x: get_year(x,docs))
res1['cont'] = res1[0].apply(lambda x: get_cont(x,docs))
res1

,0,cur_year_token,cur_all_token_cnt,word_cnt,doc_cnt,df_cnt,rate,tf,idf,score,year,cont
0,咸丰八年,17,17,9356,195,17,0.944444,0.007268,2.439786,0.016747,咸丰八年三月初六日咸丰八年三月廿四日咸丰八年三月三十日咸丰八年五月三十日咸丰八年七月二十一日...,百无一成，所以谆谆教各位弟弟引以为戒。（咸丰八年三月初六日） 致九弟·愿共
1,九弟左右,12,18,9356,195,18,0.631579,0.005130,2.382628,0.007720,咸丰七年九月二十二日咸丰七年十月初四日咸丰七年十二月十四日咸丰八年三月初六日咸丰八年三月廿四...,沅甫九弟左右：十二日申刻，代一从县里回来，接
2,甫九弟左,11,16,9356,195,16,0.647059,0.004703,2.500411,0.007609,咸丰七年九月二十二日咸丰七年十月初四日咸丰七年十二月十四日咸丰八年三月初六日咸丰八年三月廿四...,沅甫九弟左右：十二日申刻，代一从县里回来，
3,两个弊病,3,3,9356,195,1,0.750000,0.001283,5.273000,0.005072,咸丰八年三月廿四日,去职，丧失威信，心里感到内疚。长傲、多言两个弊病，以前的世卿大夫的兴与衰，以及近
4,得意的事,3,3,9356,195,2,0.750000,0.001283,4.579852,0.004406,道光元年，就处在顺境之中，经历了三十年，都非常平安。从咸丰年开始，每每碰到得意的事，便有不得...,十年，都非常平安。从咸丰年开始，每每碰到得意的事，便有不得意的事，相继而来，壬子
5,李次青的,3,3,9356,195,2,0.750000,0.001283,4.579852,0.004406,咸丰八年正月十一日咸丰八年月初四日,慢火温，弟弟现在正是用猛火攻的时候。 李次青的才能，实在赶不上，一方面稍微表示
6,克复吉安,4,5,9356,195,5,0.666667,0.001710,3.663562,0.004177,咸丰八年五月三十日道光元年，就处在顺境之中，经历了三十年，都非常平安。从咸丰年开始，每每碰到...,者合，或在新住宅，或另立规模，一概等九弟克复吉安之后回家料理。建造祠堂的方法，也
7,丰八年三,3,3,9356,195,3,0.750000,0.001283,4.174387,0.004016,咸丰八年三月初六日咸丰八年三月廿四日咸丰八年三月三十日,无一成，所以谆谆教各位弟弟引以为戒。（咸丰八年三月初六日） 致九弟·愿共鉴
8,丰八年五,3,3,9356,195,3,0.750000,0.001283,4.174387,0.004016,咸丰八年五月三十日咸丰八年五月初五日咸丰八年五月初六日,有一个框架，没有一点精彩，惭愧惭愧！（咸丰八年五月三十日） 致四弟季弟·在
9,丰八年十,3,3,9356,195,3,0.750000,0.001283,4.174387,0.004016,咸丰八年十一月二十三日道光元年，就处在顺境之中，经历了三十年，都非常平安。从咸丰年开始，每每...,夫，至今感到惭愧，所以反复强调一番。（咸丰八年十一月二十三日） 致诸弟·述


In [35]:
sort2 = cur_year_tokens[cur_year_tokens['cur_year_token']==1].sort_values(by=['score','df_cnt'],ascending=(False,True))
res2 = sort2[:50].reset_index(drop=True)
res2['year'] = res2[0].apply(lambda x: get_year(x,docs))
res2['cont'] = res2[0].apply(lambda x: get_cont(x,docs))
res2

,0,cur_year_token,cur_all_token_cnt,word_cnt,doc_cnt,df_cnt,rate,tf,idf,score,year,cont
0,一下我的,1,1,9356,195,1,0.5,0.000428,5.273,0.001127,咸丰八年正月十一日,李次青的才能，实在赶不上，一方面稍微表示一下我的歉意，一方面遇什么事情都可向他请
1,一个做一,1,1,9356,195,1,0.5,0.000428,5.273,0.001127,咸丰八年正月十一日,弟弟现在以营务为重，不适合经常看书，凡属一个做一件事，要专一才能精到，如果专一，
2,一个傲字,1,1,9356,195,1,0.5,0.000428,5.273,0.001127,咸丰八年三月初六日,改。我在军中多年，难道没有一点可取，只因一个傲字，百无一成，所以谆谆教各位弟弟引
3,一个军队,1,1,9356,195,1,0.5,0.000428,5.273,0.001127,咸丰八年三月三十日,员宠杂相处，就可预卜国家会衰败。善于观测一个军队也是如此，似乎应该区别对待；很无
4,一个刚正,1,1,9356,195,1,0.5,0.000428,5.273,0.001127,咸丰八年五月初六日,官吏的才能过于带兵的才能，将来或者可以做一个刚正廉明的官长，切实做几件对老百姓有
5,一个早晨,1,1,9356,195,1,0.5,0.000428,5.273,0.001127,咸丰八年四月初九日,，不能取胜，或者受到小挫折，那么你的名声一个早晨的时间便下落了，所以说探验的方法
6,一个框架,1,1,9356,195,1,0.5,0.000428,5.273,0.001127,咸丰八年五月三十日,，抄稿寄给你看，笔墨功夫许久荒疏了，只有一个框架，没有一点精彩，惭愧惭愧！（咸丰
7,一个表现,1,1,9356,195,1,0.5,0.000428,5.273,0.001127,咸丰八年三月廿四日,定在下次寄到你那里转交。这也是为兄长做的一个表现，弟弟既然提出来了，我岂敢还不改
8,一个通才,1,1,9356,195,1,0.5,0.000428,5.273,0.001127,咸丰八年五月三十日,林氏续讯古文雅正》，虽说一向不知名，也是一个通才。如有《大学衍义》《衍义补》两书
9,一事无成,1,1,9356,195,1,0.5,0.000428,5.273,0.001127,咸丰八年正月十一日,实在在的心愿而缺乏实现心愿的实力，一定会一事无成。李云麟的长处和短处，也和我相似
